In [130]:
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline

In [82]:
from tensorflow.examples.tutorials.mnist import input_data

In [83]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Making the Helper Functions

In [84]:
# Function to declare the tensor weights
def init_weights(shape):
    
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial_value=init_random_dist)

In [85]:
def init_bias(shape):
    
    init_bias = tf.constant(0.1, shape = shape)
    return tf.Variable(initial_value=init_bias)

In [86]:
# Function to do the convolution
def conv2by2(x, W):
    return tf.nn.conv2d(input=x,filter=W,strides=[1, 1, 1, 1],padding='SAME')

In [87]:
# Function for pooling layer
def pool2by2(x):
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [88]:
# Funciton to make the convolution layers
def convolution_layers(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    
    return tf.nn.relu(conv2by2(input_x, W) + b)

In [96]:
# Function for fully connected Neural Network Layers
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [106]:
# Making the placeholders
x = tf.placeholder(tf.float32, shape=[None,784])
y_true = tf.placeholder(tf.float32, shape=[None,10])

# Making the Layers

In [107]:
# we need to reshape every input image
x_image = tf.reshape(x, shape=[-1,28,28,1])

In [108]:
# convolution layer 1
convo_1 = convolution_layers(x_image,shape=[6,6,1,44])
convo_1_pooling = pool2by2(convo_1)

In [109]:
# convolution Layer 2
convo_2 = convolution_layers(convo_1_pooling,[4,4,44,88])
convo_2_pooling = pool2by2(convo_2)

In [110]:
# flattening the output for feeding in the ANN
convo_2_flat = tf.reshape(convo_2_pooling,shape=[-1,7*7*88])

In [111]:
# making first full layer
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [112]:
# Making the Dropout function
hold_prob = tf.placeholder(tf.float32)
full_dropout_one = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [113]:
# making the final predictions through the neural networks
y_pred = normal_full_layer(full_dropout_one,10)

# Making Loss Functions and Session

In [114]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits=y_pred))

In [115]:
# making the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [116]:
train = optimizer.minimize(cross_entropy)

In [131]:
saver = tf.train.Saver()

In [134]:
# Making the sesssion and Running
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    steps = 100
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(444)
        
        sess.run(train, feed_dict = {y_true: batch_y, hold_prob:0.5, x: batch_x})
        
        saver.save(sess,'models/My_MNIST_with_CNN.ckpt')
        
        if i%100==0:
            print("At step {}".format(i))
            print("Accuracy on testing set is: ")
            
            # testing the full test set
            evaluate = tf.equal(tf.argmax(y_true,axis=1), tf.argmax(y_pred, axis=1))
            acc = tf.reduce_mean(tf.cast(evaluate,tf.float32))
            print(sess.run(acc, feed_dict={y_true:mnist.test.labels, x:mnist.test.images, hold_prob:1.0}))
            print("\n")

At step 0
Accuracy on testing set is: 
0.2043


